In [17]:
import pandas as pd
import matplotlib as plt

In [18]:
transactions = pd.read_csv("./data/amazon-purchases.csv")

In [19]:
print(transactions.head())

   Order Date  Purchase Price Per Unit  Quantity Shipping Address State  \
0  2018-12-04                     7.98       1.0                     NJ   
1  2018-12-22                    13.99       1.0                     NJ   
2  2018-12-24                     8.99       1.0                     NJ   
3  2018-12-25                    10.45       1.0                     NJ   
4  2018-12-25                    10.00       1.0                     NJ   

                                               Title ASIN/ISBN (Product Code)  \
0  SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...               B0143RTB1E   
1  Betron BS10 Earphones Wired Headphones in Ear ...               B01MA1MJ6H   
2                                                NaN               B078JZTFN3   
3  Perfecto Stainless Steel Shaving Bowl. Durable...               B06XWF9HML   
4                      Proraso Shaving Cream for Men               B00837ZOI0   

        Category  Survey ResponseID  
0   FLASH_MEMORY  R_01vN

In [20]:
transactions["Order Date"] = pd.to_datetime(transactions["Order Date"])

## Todo 
#Most of missing values in Shipping address State can be replaced by looking at customer survey ID (customer identifactor) -> if same customer we can asume same delivery address and fill in missing values 
#-> Maybe add additonal check if there is a transaction on the same date to be 100% sure

In [21]:
# Sort Survey ResponseID & Date 
# Check if value in "Shipping Address State" == NaN
# if yes == check if row-1 same Survey ResponseID 
    # True: Fill NaN with Shipping Address State else: 0

In [22]:
transactions_sorted = transactions.sort_values(["Survey ResponseID", "Order Date"], ascending=[True, True])

In [23]:
"""#mask = transactions_sorted["Shipping Address State"].isna()

transactions_sorted["Prev Survey ResponseID"] = transactions_sorted["Survey ResponseID"].shift(1) # Create 

for i in range(1, len(transactions_sorted)):
    if isinstance(transactions_sorted["Shipping Address State"][i], float): # Check if Shipping Address is NaN, for some reason nan is saved as float
        if transactions_sorted["Survey ResponseID"][i] == transactions_sorted["Prev Survey ResponseID"][i]: 
            transactions_sorted["Shipping Address State"][i] = transactions_sorted["Shipping Address State"][i-1]
            print(i)
            #break
        elif transactions_sorted["Survey ResponseID"][i] == transactions_sorted["Prev Survey ResponseID"][i+1]:
            transactions_sorted["Shipping Address State"][i] = transactions_sorted["Shipping Address State"][i+1]
        #else: # Trying to keep it clear 
         #  if not(isinstance(transactions_sorted["Shipping Address State"][i], float)) and 
"""

'#mask = transactions_sorted["Shipping Address State"].isna()\n\ntransactions_sorted["Prev Survey ResponseID"] = transactions_sorted["Survey ResponseID"].shift(1) # Create \n\nfor i in range(1, len(transactions_sorted)):\n    if isinstance(transactions_sorted["Shipping Address State"][i], float): # Check if Shipping Address is NaN, for some reason nan is saved as float\n        if transactions_sorted["Survey ResponseID"][i] == transactions_sorted["Prev Survey ResponseID"][i]: \n            transactions_sorted["Shipping Address State"][i] = transactions_sorted["Shipping Address State"][i-1]\n            print(i)\n            #break\n        elif transactions_sorted["Survey ResponseID"][i] == transactions_sorted["Prev Survey ResponseID"][i+1]:\n            transactions_sorted["Shipping Address State"][i] = transactions_sorted["Shipping Address State"][i+1]\n        #else: # Trying to keep it clear \n         #  if not(isinstance(transactions_sorted["Shipping Address State"][i], float)) a

In [24]:
#transactions_sorted["Survey ResponseID"][140] == transactions_sorted["Prev Survey ResponseID"][140]


In [25]:
#transactions.groupby("Survey ResponseID")

In [26]:
#transactions["Shipping Address State"]

## Adding customer information

In [27]:
customers = pd.read_csv("./data/survey.csv")

### Formating Living state to match with Temperature Table
We use the living state of customer table to fill in missing values in the transaction table -> need to be in the same format

In [28]:
# Create a dictionary of state name mappings
state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Assuming your dataframe is called 'df' and the column with state names is called 'state'
customers["Q-demos-state"] = customers["Q-demos-state"].replace(state_abbrev)

### Mergin Dataframes

In [29]:
amazon_data = transactions.merge(customers, on="Survey ResponseID", how="left")

### Dealing with missing shipping addresses

In [46]:
amazon_data.loc[amazon_data["Shipping Address State"].isna(), "Shipping Address State"] = amazon_data["Q-demos-state"] # Replacing missing Shipping Addresses with living address from ordering customer

amazon_data.loc[amazon_data["Shipping Address State"] == "PR", "Shipping Address State"] = amazon_data["Q-demos-state"] # There is a shipping address state called PR which could be Puerto Rico, we replace this with the living state of the customer ordering


## Adding Temperature information

In [47]:
weather = pd.read_csv("./Data/weather_data.csv")

print(weather["st_abb"].nunique()) # No weather information for Alaska and Hawaii
weather.head()

48


,Unnamed: 0,st_abb,st_code,date,stability,ppt,tmin,tmax,tavg
0,1,AL,1,20180101,stable,1.602,-7.569,3.303,-2.133
1,2,AL,1,20180102,stable,0.002,-9.746,-0.868,-5.307
2,3,AL,1,20180103,stable,0.000,-9.822,2.161,-3.830
3,4,AL,1,20180104,stable,0.000,-5.802,4.918,-0.442
4,5,AL,1,20180105,stable,0.013,-8.308,2.945,-2.681


In [48]:
weather["date"] = pd.to_datetime(weather["date"], format="%Y%m%d")
weather.rename(columns={"st_abb": "Shipping Address State"}, inplace=True)
weather.head()

,Unnamed: 0,Shipping Address State,st_code,date,stability,ppt,tmin,tmax,tavg
0,1,AL,1,2018-01-01,stable,1.602,-7.569,3.303,-2.133
1,2,AL,1,2018-01-02,stable,0.002,-9.746,-0.868,-5.307
2,3,AL,1,2018-01-03,stable,0.000,-9.822,2.161,-3.830
3,4,AL,1,2018-01-04,stable,0.000,-5.802,4.918,-0.442
4,5,AL,1,2018-01-05,stable,0.013,-8.308,2.945,-2.681


In [49]:
# Joining weather

data_combined = pd.merge(
    left=amazon_data, 
    right=weather,
    how='left',
    left_on=['Shipping Address State', 'Order Date'],
    right_on=['Shipping Address State', 'date'],
)

In [50]:
data_grouped = data_combined.groupby("Shipping Address State")
data_grouped["tavg"].count()

Shipping Address State
AK                                            0
AL                                        18830
AR                                        14859
AZ                                        34273
CA                                       187520
CO                                        30697
CT                                        18021
DC                                            0
DE                                         7723
FL                                       115704
GA                                        56123
HI                                            0
I did not reside in the United States         0
IA                                        11441
ID                                         6528
IL                                        76102
IN                                        46960
KS                                        16386
KY                                        28466
LA                                        20973
MA               

## CHECKPOINT Saving complete combined dataset

In [51]:
data_combined.to_csv('./data/data_combined.csv')

In [ ]:
""""Fashion = [
    "ACCESSORY", "ADULT_COSTUME", "APPAREL", "APPAREL_BELT", "APPAREL_GLOVES",
    "APPAREL_HEAD_NECK_COVERING", "APPAREL_PIN", "APPENDAGE_WARMER", "APRON",
    "ARM_SLEEVE", "Apparel", "BABY_JUMPER_WALKER", "BELTS", "BLAZER",
    "BLUE_LIGHT_BLOCKING_EYEGLASSES", "BODYSTOCKING", "BOOT", "BRA", "BRACELET",
    "BRA_UNDERWEAR_SET", "CHOLI", "COAT", "COORDINATED_OUTFIT", "CORRECTIVE_EYEGLASSES",
    "CORSET", "DRESS", "DUPATTA", "EARMUFF", "EARRING", "ETHNIC_WEAR",
    "FASHIONEARRING", "FASHIONNECKLACEBRACELETANKLET", "FASHIONOTHER", "FASHIONRING",
    "FASHION_JEWELRY", "FINEEARRING", "FINENECKLACEBRACELETANKLET", "FINERING",
    "FOOTWEAR", "GUILD_APPAREL", "GUILD_JEWELRY", "GUILD_SHOES", "HAT", "HOSIERY",
    "JEWELRY", "JEWELRY_SET", "KIMONO", "KURTA", "LEG_SLEEVE", "LEOTARD",
    "MOISTURIZING_SOCK_GLOVE", "NECKLACE", "NECKTIE", "NIGHTGOWN_NIGHTSHIRT",
    "ORCA_SHIRT", "OUTERWEAR", "OVERALLS", "PAJAMAS", "PANTS", "PIERCING_JEWELRY",
    "ROBE", "SANDAL", "SAREE", "SHIRT", "SHOES", "SHORTS", "SKIRT", "SLEEPWEAR",
    "SLIPPER", "SNOWSHOE", "SNOWSUIT", "SNOW_PANT", "SOCK", "SOCKSHOSIERY", "SUIT",
    "SUNGLASSES", "SUSPENDER", "SWEATER", "SWEATSHIRT", "SWIMWEAR", "SWIM_CAP",
    "TIGHTS", "TRACK_SUIT", "UNDERPANTS", "UNDERWEAR", "UNDERGARMENT_SLIP",
    "UNDERGARMENT_THIGH_SLIMMER", "UNION_SUIT", "VEST", "WATCH", "WATCHES", "WATCH_BAND"
]


Groceries = [
   "ALCOHOLIC_BEVERAGE", "BABY_FOOD", "BABY_FORMULA", "BAKING_CHOCOLATE", "BEER",
   "BEVERAGE", "BREAD", "BREAKFAST_CEREAL", "CAKE", "CANDY", "CEREAL",
   "CHOCOLATE_CANDY", "COFFEE", "CONDIMENT", "COOKIE", "CRACKER", "CULINARY_SALT",
   "DAIRY_BASED_BUTTER", "DAIRY_BASED_CHEESE", "DAIRY_BASED_CREAM",
   "DAIRY_BASED_DRINK", "DAIRY_BASED_ICE_CREAM", "DAIRY_BASED_PUDDING",
   "DAIRY_BASED_YOGURT", "DONUT", "DRINK_FLAVORED", "EDIBLE_OIL_VEGETABLE",
   "FLOUR", "FRUIT", "FRUIT_SNACK", "FUDGE", "GOURMET_FOOD", "GROCERY", "Grocery",
   "HEALTH_FOOD", "HONEY", "JERKY", "JUICE_AND_JUICE_DRINK", "LEGUME",
   "MEAL_REPLACEMENT_BEVERAGE", "MEAT_ALTERNATIVE", "MILK_SUBSTITUTE",
   "NON_DAIRY_CHEESE", "NON_DAIRY_CREAM", "NON_DAIRY_ICE_CREAM",
   "NON_DAIRY_PUDDING", "NON_DAIRY_YOGURT", "NOODLE", "NUTS", "NUT_AND_SEED",
   "NUT_BUTTER", "OLIVE", "PACKAGED_SOUP_AND_STEW", "PASTRY", "POPCORN",
   "PRETZEL", "PROTEIN_DRINK", "PROTEIN_SUPPLEMENT_POWDER", "PUFFED_SNACK",
   "RICE_MIX", "RICE_WINE", "SALAD_DRESSING", "SEAFOOD", "SEASONING",
   "SNACK_CHIP_AND_CRISP", "SNACK_FOOD", "SNACK_FOOD_BAR", "SNACK_MIX",
   "SPORTS_DRINK", "SUGAR", "SUGAR_CANDY", "SUGAR_SUBSTITUTE", "SYRUP", "TEA",
   "TOFU", "VEGETABLE", "VEGETARIAN_EGG_SUBSTITUTE", "WATER"
]""""

SyntaxError: unterminated string literal (detected at line 40) (1808289747.py, line 40)

## Filtering for the Categories

In [ ]:
"""mask_fashion = amazon_data["Category"].isin(Fashion)
Fashion_transactions = amazon_data[mask_fashion]


mask_groceries = amazon_data["Category"].isin(Groceries)
Grocerie_transactions = amazon_data[mask_groceries]"""

In [ ]:
"""print(Fashion_transactions["Category"].nunique()) # One Category seems to be missing (maybe typo), or upper and lower letters
print(Grocerie_transactions["Category"].nunique())"""

91
78


In [ ]:
"""print(f"Missing Values Fashion:\n{Fashion_transactions.isna().sum()}")
print(f"Missing Values Groceries:\n{Grocerie_transactions.isna().sum()}")"""

Missing Values Fashion:
Order Date                         0
Purchase Price Per Unit            0
Quantity                           0
Shipping Address State          3765
Title                             47
ASIN/ISBN (Product Code)          94
Category                           0
Survey ResponseID                  0
Q-demos-age                        0
Q-demos-hispanic                   0
Q-demos-race                       0
Q-demos-education                  0
Q-demos-income                     0
Q-demos-gender                     0
Q-sexual-orientation               0
Q-demos-state                      0
Q-amazon-use-howmany               0
Q-amazon-use-hh-size               0
Q-amazon-use-how-oft               0
Q-substance-use-cigarettes         0
Q-substance-use-marijuana          0
Q-substance-use-alcohol            0
Q-personal-diabetes                0
Q-personal-wheelchair              0
Q-life-changes                106531
Q-sell-YOUR-data                   0
Q-sell-consume